In [ ]:
base_path = "/content/drive/MyDrive/UFSC/INE5448"

# Treinando LLM

### Dependências

In [ ]:
!pip install -U transformers peft accelerate bitsandbytes unsloth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.7/358.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 140.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: torchao
    F

In [ ]:
# 1. Desinstalar versões conflitantes
!pip uninstall unsloth torchao -y

In [ ]:


# 2. Reinstalar Unsloth forçando a versão mais atual compatível com o Colab
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 3. Instalar dependências opcionais sem dependências cruzadas (para não quebrar o torch)
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


ERROR: Operation cancelled by user


### Carregando modelo pré-treinado

In [ ]:
import unsloth

model, tokenizer = unsloth.FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

### Preparando Dataset

Está formando pares entre textos que o OCR extraiu e o ground truth do mesmo, formando os datasets de treino e teste

In [ ]:
import json
import random

# --- CONFIGURAÇÃO ---
INPUT_FILE = base_path + "/images_family_search/ocr_curadoria.json"
OUTPUT_TRAIN = base_path + "/images_family_search/train_dataset.jsonl"
OUTPUT_TEST = base_path + "/images_family_search/test_dataset.jsonl"
TEST_SIZE = 5  # Quantas imagens guardaremos para a demo final

SYSTEM_PROMPT = (
    "Você é um assistente especializado em pós-processamento de OCR para documentos manuscritos históricos. "
    "Sua tarefa é corrigir erros ortográficos e de leitura gerados pelo OCR, baseando-se no contexto. "
    "Mantenha nomes próprios, datas e locais exatamente como no original. "
    "Não invente informações que não estão no texto. "
    "Se o texto estiver cortado ou incompleto, corrija apenas o que é visível."
)

def preparar_datasets_finais():
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    valid_entries = []

    # 1. Filtra apenas os marcados como 'usable'
    for item in data:
        if item.get('usable') is True:
            ocr_input = item['ocr_text']
            # Remove quebras de linha do Ground Truth para a LLM aprender a fluidez
            ground_truth = item['ground_truth'].replace('\n', ' ').strip()

            # Formato ChatML
            entry = {
                "messages": [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Corrija este texto OCR:\n{ocr_input}"},
                    {"role": "assistant", "content": ground_truth}
                ],
                # Guardamos metadados para você saber qual imagem é qual depois
                "metadata": {
                    "filename": item['filename'],
                    "ocr_original": ocr_input
                }
            }
            valid_entries.append(entry)

    total = len(valid_entries)
    print(f"Total de itens válidos encontrados: {total}")

    if total < 10:
        print("⚠️ Cuidado: Pouquíssimos dados. Tente corrigir pelo menos 20.")

    # 2. Embaralhar e Separar
    random.seed(42) # Seed fixa para reproduzibilidade
    random.shuffle(valid_entries)

    test_data = valid_entries[:TEST_SIZE]
    train_data = valid_entries[TEST_SIZE:]

    # 3. Salvar Treino (JSONL puro para Unsloth)
    with open(OUTPUT_TRAIN, 'w', encoding='utf-8') as f:
        for entry in train_data:
            # Remove metadados do arquivo de treino para não confundir a lib
            clean_entry = {"messages": entry["messages"]}
            json.dump(clean_entry, f, ensure_ascii=False)
            f.write('\n')

    # 4. Salvar Teste (Com metadados para você usar na demo)
    with open(OUTPUT_TEST, 'w', encoding='utf-8') as f:
        # Salvamos como JSON normal (lista) para facilitar leitura visual
        json.dump(test_data, f, ensure_ascii=False, indent=4)

    print(f"✅ CONCLUÍDO!")
    print(f"📁 {len(train_data)} exemplos salvos em '{OUTPUT_TRAIN}' (Use para treinar a IA).")
    print(f"🧪 {len(test_data)} exemplos salvos em '{OUTPUT_TEST}' (Use para validar e gravar o vídeo).")

# Rode apenas quando terminar as 50 correções
preparar_datasets_finais()

Total de itens válidos encontrados: 50
✅ CONCLUÍDO!
📁 45 exemplos salvos em '/content/drive/MyDrive/UFSC/INE5448/images_family_search/train_dataset.jsonl' (Use para treinar a IA).
🧪 5 exemplos salvos em '/content/drive/MyDrive/UFSC/INE5448/images_family_search/test_dataset.jsonl' (Use para validar e gravar o vídeo).


### Configurando Trainer

Está determinando hiperparâmetros e ajustando o prompt para desconsiderar (mask) o texto do OCR durante a avaliação. Em outras palavras, está treinando apenas a correção do texto.

In [ ]:
import os
import torch
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
from transformers import DataCollatorForSeq2Seq
from unsloth import FastLanguageModel

# --- 1. Configuração Inicial e Dataset ---

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,

    lora_dropout = 0.05,  # 10% de chance de esquecer neurônios (evita vício)
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
max_seq_length = 2048
# Configura o dataset
OUTPUT_TRAIN = base_path + "/images_family_search/train_dataset.jsonl"
OUTPUT_TEST = base_path + "/images_family_search/test_dataset.jsonl"
OUTPUT_DIR = base_path + "/outputs_llama3_ocr"

# --- 2. Carregar o Dataset que criamos ---
# Nota: Como temos poucos dados, não vamos usar validação durante o treino para não perder dados de treino.
# Usaremos o arquivo de teste visualmente depois.
dataset = load_dataset("json", data_files={"train": OUTPUT_TRAIN})

# --- 3. Pré-processamento Inteligente (A adaptação do seu código) ---
def preprocess_and_mask(example):
    # O dataset vem como lista de mensagens. Vamos extrair o texto.
    messages = example['messages']

    # Extrai o Input (System + User) e o Output (Assistant)
    # Formato Llama 3 Chat Template manual
    system_msg = messages[0]['content']
    user_msg = messages[1]['content']
    assistant_msg = messages[2]['content']

    # Monta o Prompt no estilo Llama 3
    prompt_text = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    response_text = f"{assistant_msg}<|eot_id|>"

    # 1. Tokeniza
    prompt_ids = tokenizer.encode(prompt_text, add_special_tokens=False)
    response_ids = tokenizer.encode(response_text, add_special_tokens=False)

    # 2. Concatena
    input_ids = prompt_ids + response_ids
    attention_mask = [1] * len(input_ids)

    # 3. Cria os Labels (Máscara no Prompt = -100)
    # Isso é o que faz seu modelo ficar BOM com poucos dados!
    labels = [-100] * len(prompt_ids) + response_ids

    # Truncamento
    if len(input_ids) > max_seq_length:
        input_ids = input_ids[:max_seq_length]
        attention_mask = attention_mask[:max_seq_length]
        labels = labels[:max_seq_length]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

print("Processando e tokenizando...")
train_dataset = dataset["train"].map(preprocess_and_mask)

# --- 3. Data Collator ---
# Usamos o Collator de Seq2Seq porque ele sabe lidar com padding de Labels usando -100
# (Isso funciona perfeitamente para Decoder-only também quando os labels já existem)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)

# --- 4. Configuração do Trainer (Limpa) ---
sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2, # Batch pequeno para GPU grátis
    gradient_accumulation_steps=4, # Simula batch 8
    warmup_steps=5,
    # Aumentei as épocas porque temos MUITO poucos dados (40-50).
    # Precisamos repetir para ele aprender.
    num_train_epochs=30,
    learning_rate=2e-4, # Taxa padrão para QLoRA (a sua estava 2e-5, muito lenta para MVP)
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    report_to="none" # Desliga WandB para não pedir login
)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="input_ids", # Dummy field, já processamos manualmente
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    args=sft_config,
)

trainer.train()


Unsloth: Already have LoRA adapters! We shall skip this step.


Processando e tokenizando...


Map:   0%|          | 0/45 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 45 | Num Epochs = 30 | Total steps = 180
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.578900
2,2.569300
3,1.729400
4,2.517700
5,2.010900
6,1.685600
7,1.962100
8,1.835700
9,1.412400
10,1.295700


TrainOutput(global_step=180, training_loss=0.18315959255883676, metrics={'train_runtime': 683.1751, 'train_samples_per_second': 1.976, 'train_steps_per_second': 0.263, 'total_flos': 1.660416552787968e+16, 'train_loss': 0.18315959255883676, 'epoch': 30.0})

In [ ]:
# --- INFERÊNCIA / TESTE FINAL ---
import json
from unsloth import FastLanguageModel

OUTPUT_TEST = base_path + "/images_family_search/test_dataset.jsonl"
LLAMA_DIR = base_path + "/outputs_llama3_ocr"

# 1. Carrega o modelo treinado (se já não estiver na memória)
# Se você acabou de treinar e a variável 'model' ainda existe, pule esta etapa.
# Caso tenha reiniciado:
if 'model' not in locals():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = LLAMA_DIR + "/checkpoint-180", # Pasta onde salvamos
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Otimiza para gerar texto

# 2. Carrega os dados de teste (que o modelo NUNCA viu)
with open(OUTPUT_TEST, "r", encoding="utf-8") as f:
    test_data = json.load(f)

print(f"🧪 Testando em {len(test_data)} imagens inéditas...\n")

data = []

for i, item in enumerate(test_data):
    # Recupera o OCR sujo
    # A estrutura salva no JSON de teste era a lista completa de mensagens
    # O user message é o índice 1
    ocr_input_msg = item['messages'][1]['content']

    # Extrai só o texto do OCR (remove "Corrija este texto OCR:\n")
    ocr_text_only = ocr_input_msg.replace("Corrija este texto OCR:\n", "")

    # O Ground Truth (para compararmos se acertou)
    ground_truth = item['messages'][2]['content']

    # 3. Monta o Prompt (Formato Llama 3)
    system_prompt = (
    "Você é um corretor de OCR estrito. Sua única função é corrigir erros de digitação. "
    "NÃO altere nomes próprios. NÃO complete frases com informações ausentes. "
    "Se o OCR estiver muito ruim, mantenha o texto original ou aproxime-se do som das letras. "
    "Seja fiel ao texto de entrada."
    )
    # system_prompt = item['messages'][0]['content']

    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCorrija este texto OCR:\n{ocr_text_only}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    # 4. Gera a resposta
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        use_cache=True,
        # MUDANÇAS AQUI:
        temperature=0.4,       # Aumenta um pouco a criatividade para ele não repetir frases decoradas
        repetition_penalty=1.1, # Penaliza repetição de padrões
        do_sample=True,
        top_p=0.9,
    )

    # Decodifica
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Limpeza da string gerada (remove o prompt que vem junto)
    # O Llama 3 repete o prompt, pegamos só o final
    try:
        response_clean = prediction.split("assistant\n\n")[-1].strip()
    except:
        response_clean = prediction

    data.append({"id": i,
                 "ocr": ocr_text_only,
                 "llm": response_clean,
                 "gt": ground_truth})
    # 5. Mostra o Resultado
    print(f"--- AMOSTRA {i+1} ---")
    print(f"📥 OCR SUJO:     {ocr_text_only}")
    print(f"🤖 LLM PREVISTO: {response_clean}")
    print(f"✅ GABARITO:     {ground_truth}")
    print("-" * 50)

🧪 Testando em 5 imagens inéditas...

--- AMOSTRA 1 ---
📥 OCR SUJO:     ré de Seabra ja fättecido e Altaria
das Dores do Espirito Santo. Com fir
muza do que eu josé Pacheco dect
buquerque Maranha's lavroesten
Vermo do que vai
assignados
por todos.
Mil Jaer de Andre Radeber
🤖 LLM PREVISTO: -recebo este termo como lido e conforme val por todos. Mano do que eu José Pacheco de Albuquerque Maranhão, lavrei este termo no lugar onde vai assignado por todos. Mil e treze de Andre Rademaher
✅ GABARITO:     -ré de Jeabra, ja fallecido e Maria das Dores do Espirito Santo. Em firmeza do que eu josé Pacheco de Albuquerque Maranhão, lavro este termo do que vai assignados por todos. Maggler de Andre de Jeabra
--------------------------------------------------
--- AMOSTRA 2 ---
📥 OCR SUJO:     vão do seu Curgo, e as testemunhur
abaixo assignados, receberam-se
em Matrimonio João Pedro Car
neiro, e Rosa Mária de Espirito San
🤖 LLM PREVISTO: -ras no Estado de seu natural, e as testemunhos abaixo assignados

In [ ]:
!pip install jiwer

In [ ]:


import jiwer
import pandas as pd

# --- FUNÇÃO DE PRÉ-PROCESSAMENTO ---
# Definimos a transformação
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemovePunctuation(),
    jiwer.Strip(),
])

print(f"{'ID':<3} | {'Métrica':<10} | {'OCR (Original)':<15} | {'LLM (Ours)':<15} | {'Delta':<10}")
print("-" * 65)

avg_wer_ocr = 0
avg_wer_llm = 0
avg_cer_ocr = 0
avg_cer_llm = 0

for item in data:
    # 1. Limpeza básica manual (\n vira espaço)
    ocr_raw = item['ocr'].replace('\n', ' ')
    llm_raw = item['llm'].replace('\n', ' ')
    gt_raw = item['gt'].replace('\n', ' ')

    # 2. Aplicar transformação do Jiwer ANTES de calcular
    # Isso resolve o erro de versão
    ocr_clean = transformation(ocr_raw)
    llm_clean = transformation(llm_raw)
    gt_clean = transformation(gt_raw)

    # Calcula WER (Agora passamos apenas as strings já limpas)
    wer_ocr = jiwer.wer(gt_clean, ocr_clean)
    wer_llm = jiwer.wer(gt_clean, llm_clean)

    # Calcula CER
    cer_ocr = jiwer.cer(gt_clean, ocr_clean)
    cer_llm = jiwer.cer(gt_clean, llm_clean)

    # Acumula médias
    avg_wer_ocr += wer_ocr
    avg_wer_llm += wer_llm
    avg_cer_ocr += cer_ocr
    avg_cer_llm += cer_llm

    print(f"{item['id']:<3} | WER        | {wer_ocr:.2%}          | {wer_llm:.2%}          | {wer_llm - wer_ocr:+.2%}")
    print(f"{'':<3} | CER        | {cer_ocr:.2%}          | {cer_llm:.2%}          | {cer_llm - cer_ocr:+.2%}")
    print("-" * 65)

# Médias Finais
n = len(data)
print(f"\n=== MÉDIAS GERAIS ===")
print(f"WER Médio OCR (Baseline): {avg_wer_ocr/n:.2%}")
print(f"WER Médio LLM (Ours):     {avg_wer_llm/n:.2%}")
print(f"CER Médio OCR (Baseline): {avg_cer_ocr/n:.2%}")
print(f"CER Médio LLM (Ours):     {avg_cer_llm/n:.2%}")

ID  | Métrica    | OCR (Original)  | LLM (Ours)      | Delta     
-----------------------------------------------------------------
0   | WER        | 44.44%          | 63.89%          | +19.44%
    | CER        | 15.90%          | 42.05%          | +26.15%
-----------------------------------------------------------------
1   | WER        | 19.05%          | 33.33%          | +14.29%
    | CER        | 3.17%          | 15.87%          | +12.70%
-----------------------------------------------------------------
2   | WER        | 40.00%          | 40.00%          | +0.00%
    | CER        | 18.01%          | 20.85%          | +2.84%
-----------------------------------------------------------------
3   | WER        | 54.17%          | 54.17%          | +0.00%
    | CER        | 25.19%          | 30.37%          | +5.19%
-----------------------------------------------------------------
4   | WER        | 43.24%          | 51.35%          | +8.11%
    | CER        | 18.34%          | 27.95%